<a href="https://colab.research.google.com/github/ADRIANATORREZANI/ADRIANATORREZANI/blob/main/PCA_An%C3%A1lise_dos_Componentes_Principais_Adriana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. Importação de bibliotecas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.decomposition import PCA

# --- 2. Carregamento dos dados ---
url_portugues = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQmEm4SuBKE2_dXddpxAM38P7YbLWcRIAcb5lqql9nKlw4pbNd4sZS__a6oK1mI3MDfQu8PvHEUJVMC/pub?gid=998950341&single=true&output=csv'
url_matematica = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQmEm4SuBKE2_dXddpxAM38P7YbLWcRIAcb5lqql9nKlw4pbNd4sZS__a6oK1mI3MDfQu8PvHEUJVMC/pub?gid=1209425280&single=true&output=csv'

portugues = pd.read_csv(url_portugues)
matematica = pd.read_csv(url_matematica)

# --- 3. Tratamento de valores nulos ---
def tratar_nulos(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('não informado')
        else:
            df[col] = df[col].fillna(df[col].median())
    return df

portugues = tratar_nulos(portugues)
matematica = tratar_nulos(matematica)

# --- 4. Pré-processamento ---
def preprocessamento(df):
    df_proc = df.copy()
    le = LabelEncoder()
    for col in df_proc.columns:
        if df_proc[col].dtype == 'object':
            df_proc[col] = le.fit_transform(df_proc[col].astype(str))
    df_proc = df_proc.fillna(0)
    return df_proc

portugues_proc = preprocessamento(portugues)
matematica_proc = preprocessamento(matematica)

# ----------------------------
# FUNÇÃO PARA ANALISAR UM DATASET (Português ou Matemática)
# ----------------------------
def analisar_disciplina(df_proc, nome_disciplina="Disciplina"):
    print(f"\n===== ANÁLISE - {nome_disciplina.upper()} =====")

    # Regressão sem PCA
    X = df_proc.drop(['nota_final'], axis=1)
    y = df_proc['nota_final']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    print("\nRegressão Linear - Sem PCA")
    print("R²:", r2_score(y_test, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

    # Classificação sem PCA
    df_proc['Aprovado'] = np.where(df_proc['nota_final'] >= 10, 1, 0)
    X_class = df_proc.drop(['nota_final', 'Aprovado'], axis=1)
    y_class = df_proc['Aprovado']
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

    logr = LogisticRegression(max_iter=1000)
    logr.fit(X_train_c, y_train_c)
    y_pred_logr = logr.predict(X_test_c)

    print("\nRegressão Logística - Sem PCA")
    print("Acurácia:", accuracy_score(y_test_c, y_pred_logr))
    print("Precisão:", precision_score(y_test_c, y_pred_logr))
    print("Recall:", recall_score(y_test_c, y_pred_logr))
    print("Matriz de Confusão:\n", confusion_matrix(y_test_c, y_pred_logr))

    # PCA
    escalador = StandardScaler()
    dados_padronizados = escalador.fit_transform(X)

    pca = PCA(n_components=0.95)
    dados_pca = pca.fit_transform(dados_padronizados)

    print(f"\nNúmero de componentes escolhidos para manter 95% da variância: {pca.n_components_}")
    print("Variância explicada por cada componente:")
    print(pca.explained_variance_ratio_)

    df_pca = pd.DataFrame(data=dados_pca, columns=[f'Componente {i+1}' for i in range(pca.n_components_)])

    # Regressão com PCA
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(df_pca, y, test_size=0.3, random_state=42)
    lr.fit(X_train_pca, y_train)
    y_pred_pca = lr.predict(X_test_pca)

    print("\nRegressão Linear - Com PCA")
    print("R²:", r2_score(y_test, y_pred_pca))
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_pca)))

    # Classificação com PCA
    y_class = df_proc['Aprovado']
    X_train_cpca, X_test_cpca, y_train_c, y_test_c = train_test_split(df_pca, y_class, test_size=0.3, random_state=42)
    logr.fit(X_train_cpca, y_train_c)
    y_pred_logr_pca = logr.predict(X_test_cpca)

    print("\nRegressão Logística - Com PCA")
    print("Acurácia:", accuracy_score(y_test_c, y_pred_logr_pca))
    print("Precisão:", precision_score(y_test_c, y_pred_logr_pca))
    print("Recall:", recall_score(y_test_c, y_pred_logr_pca))
    print("Matriz de Confusão:\n", confusion_matrix(y_test_c, y_pred_logr_pca))

# --- Análises ---
analisar_disciplina(portugues_proc, "Português")
analisar_disciplina(matematica_proc, "Matemática")



===== ANÁLISE - PORTUGUÊS =====

Regressão Linear - Sem PCA
R²: 0.8699049941859338
RMSE: 1.1994205111292888

Regressão Logística - Sem PCA
Acurácia: 0.9230769230769231
Precisão: 0.9529411764705882
Recall: 0.9585798816568047
Matriz de Confusão:
 [[ 18   8]
 [  7 162]]

Número de componentes escolhidos para manter 95% da variância: 28
Variância explicada por cada componente:
[0.11961441 0.07975704 0.05542864 0.04735767 0.04451793 0.04092949
 0.03961165 0.03940761 0.03509851 0.03337025 0.03222352 0.031072
 0.02991894 0.02912885 0.02846884 0.02724144 0.02678849 0.02542356
 0.02294465 0.0225712  0.02190257 0.02096518 0.02061161 0.01988259
 0.019337   0.01784553 0.01583322 0.01512621]

Regressão Linear - Com PCA
R²: 0.8340860107745249
RMSE: 1.3545113912766873

Regressão Logística - Com PCA
Acurácia: 0.9230769230769231
Precisão: 0.9529411764705882
Recall: 0.9585798816568047
Matriz de Confusão:
 [[ 18   8]
 [  7 162]]

===== ANÁLISE - MATEMÁTICA =====

Regressão Linear - Sem PCA
R²: 0.7947708

In [ ]:
# 1. Importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 2. Carregando os dados das planilhas de Português e Matemática
url_portugues = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQmEm4SuBKE2_dXddpxAM38P7YbLWcRIAcb5lqql9nKlw4pbNd4sZS__a6oK1mI3MDfQu8PvHEUJVMC/pub?gid=998950341&single=true&output=csv'
url_matematica = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQmEm4SuBKE2_dXddpxAM38P7YbLWcRIAcb5lqql9nKlw4pbNd4sZS__a6oK1mI3MDfQu8PvHEUJVMC/pub?gid=1209425280&single=true&output=csv'

df_portugues = pd.read_csv(url_portugues)
df_matematica = pd.read_csv(url_matematica)

# 3. Encontrar colunas numéricas comuns
colunas_comuns = list(set(df_portugues.columns).intersection(df_matematica.columns))
colunas_numericas = [col for col in colunas_comuns if pd.api.types.is_numeric_dtype(df_portugues[col]) and pd.api.types.is_numeric_dtype(df_matematica[col])]

# 4. Criar o DataFrame unificado com média entre as duas disciplinas
dados = pd.DataFrame()
for col in colunas_numericas:
    dados[col] = (df_portugues[col] + df_matematica[col]) / 2

# 5. Tratar valores ausentes (com a média de cada coluna)
dados.fillna(dados.mean(), inplace=True)

# 6. Padronizar os dados
escalador = StandardScaler()
dados_padronizados = escalador.fit_transform(dados)

# 7. Aplicar PCA mantendo 95% da variância
pca = PCA(n_components=0.95)
dados_pca = pca.fit_transform(dados_padronizados)

# 8. Exibir número de componentes escolhidos
print(f"Número de componentes escolhidos para manter 95% da variância: {pca.n_components_}")

# 9. Variância explicada por cada componente
print("\nVariância explicada por cada componente:")
print(pca.explained_variance_ratio_)

# 10. Exibir os dados transformados
df_pca = pd.DataFrame(data=dados_pca, columns=[f'Componente {i+1}' for i in range(pca.n_components_)])
print("\nDados transformados:")
print(df_pca.head())


Número de componentes escolhidos para manter 95% da variância: 13

Variância explicada por cada componente:
[0.22411211 0.12835532 0.09566909 0.08173497 0.06571457 0.06267002
 0.05721958 0.05590501 0.04829477 0.0473516  0.04129697 0.03680304
 0.02237662]

Dados transformados:
   Componente 1  Componente 2  Componente 3  Componente 4  Componente 5  \
0     -2.466888     -0.384551      2.928476     -1.803778      2.995949   
1     -2.627991     -3.526918      0.374456      1.095064      1.213882   
2     -2.528168     -1.569741     -1.243742      0.144809     -2.014308   
3      3.914558     -1.798795      0.548906     -1.312815     -2.150490   
4      0.465593     -1.094633      2.331641     -0.210871     -0.971962   

   Componente 6  Componente 7  Componente 8  Componente 9  Componente 10  \
0     -1.234373      0.951573      1.784064      1.544837      -0.525821   
1     -0.404385     -0.066545     -0.271040     -2.402368      -0.022909   
2      1.362904     -1.532094     -1.366165 

In [ ]:
df = dados
display(df)

,saidas,idade,num_faltas,nota_periodo1,tempo_viagem,nota_periodo2,qualidade_rel_familiar,nota_final,educ_pai,consumo_alcool_fimsemana,educ_mae,estado_saude,consumo_alcool_dia,tempo_estudo,tempo_livre,num_reprovacoes
0,4.000000,18.000000,5.000000,2.500000,2.000000,8.500000,4.000000,8.500000,4.000000,1.000000,4.000000,3.000000,1.000000,2.000000,3.000000,0.000000
1,3.000000,17.000000,3.000000,7.000000,1.000000,8.000000,5.000000,8.500000,1.000000,1.000000,1.000000,3.000000,1.000000,2.000000,3.000000,0.000000
2,2.000000,15.000000,8.000000,9.500000,1.000000,10.500000,4.000000,11.000000,1.000000,3.000000,1.000000,3.000000,2.000000,2.000000,3.000000,1.500000
3,2.000000,15.000000,1.000000,14.500000,1.000000,14.000000,3.000000,14.500000,2.000000,1.000000,4.000000,5.000000,1.000000,3.000000,2.000000,0.000000
4,2.000000,16.000000,2.000000,8.500000,1.000000,11.500000,4.000000,11.500000,3.000000,2.000000,3.000000,5.000000,1.000000,2.000000,3.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,3.127848,16.627848,4.882278,11.456962,1.446835,11.374684,3.949367,11.437975,2.502532,2.267089,2.737975,3.578481,1.470886,2.021519,3.207595,0.249367
645,3.127848,16.627848,4.882278,11.456962,1.446835,11.374684,3.949367,11.437975,2.502532,2.267089,2.737975,3.578481,1.470886,2.021519,3.207595,0.249367
646,3.127848,16.627848,4.882278,11.456962,1.446835,11.374684,3.949367,11.437975,2.502532,2.267089,2.737975,3.578481,1.470886,2.021519,3.207595,0.249367
647,3.127848,16.627848,4.882278,11.456962,1.446835,11.374684,3.949367,11.437975,2.502532,2.267089,2.737975,3.578481,1.470886,2.021519,3.207595,0.249367
